In [1]:
from ethnicitydetector import EthnicityDetector
import pandas as pd
from multiprocessing import Pool
import numpy as np
import time

In [2]:
ed = EthnicityDetector()._create_ethnic_dicts()

In [3]:
for n in ed.ETHNICITY_LIST:
    print(n)

indian
japanese
greek
arabic
turkish
thai
vietnamese
balkan
italian
samoan
hawaiian
khmer
chinese
korean


In [4]:
d = pd.read_csv("/Users/ik/Data/customers/all-customers-18092017.csv.gz", encoding='latin-1',dtype=str, error_bad_lines=False)
d["full_name"] = d["first_name"].fillna('') + ' ' + d["middle_name"].fillna('') + ' ' + d["last_name"].fillna('')

In [5]:
b = d

# t0 = time.time()
# b["ethnicity"] = b["full_name"].apply(ed.get_ethnicity)
# print("elapsed time: {:.0f} min {:.0f} sec".format(*divmod(time.time() - t0, 60)))

vf = np.vectorize(ed.get_ethnicity)

def ethni(b):  
    #print("shape(b=)",b.shape)
    ets = vf(b[:,-1])
    #print(ets[:10])
    #print(type(ets))
    stk = np.hstack((b[:,0].reshape(b.shape[0],1), ets.reshape(b.shape[0],1)))
    #print(stk[:5,:])
    return stk

t0 = time.time()

pool = Pool(2)
res = pd.DataFrame(np.vstack(pool.map(ethni, np.array_split(b.loc[:, ["cust_id","full_name"]].values, 2))),
                   columns=["CustomerID", "Ethnicity"], dtype=str)
pool.close()
pool.join()

print("elapsed time: {:.0f} min {:.0f} sec".format(*divmod(time.time() - t0, 60)))

elapsed time: 7 min 17 sec


In [6]:
from collections import Counter

In [7]:
Counter(res.Ethnicity).most_common(30)

[('None', 17203181),
 ('korean', 334702),
 ('chinese', 305271),
 ('italian', 292394),
 ('indian', 209432),
 ('chinese|korean', 178657),
 ('arabic', 107944),
 ('greek', 94229),
 ('japanese', 88886),
 ('balkan', 84376),
 ('vietnamese', 80968),
 ('turkish', 42159),
 ('arabic|indian', 27677),
 ('arabic|indian|turkish', 21392),
 ('arabic|turkish', 14142),
 ('chinese|korean|vietnamese', 9323),
 ('hawaiian', 8207),
 ('thai', 6047),
 ('korean|vietnamese', 5578),
 ('khmer', 5509),
 ('samoan', 4891),
 ('japanese|korean', 4578),
 ('indian|italian', 2889),
 ('chinese|vietnamese', 2877),
 ('chinese|thai', 2743),
 ('chinese|japanese', 2211),
 ('indian|japanese', 1789),
 ('chinese|japanese|korean', 1724),
 ('arabic|italian', 1715),
 ('arabic|japanese', 1699)]

In [8]:
t0 = time.time()

b["Ethnicity"] = b["full_name"].apply(ed.get_ethnicity)

print("elapsed time: {:.0f} min {:.0f} sec".format(*divmod(time.time() - t0, 60)))

elapsed time: 9 min 27 sec


In [9]:
Counter(b.Ethnicity).most_common(15)

[(None, 17203181),
 ('korean', 334702),
 ('chinese', 305271),
 ('italian', 292394),
 ('indian', 209432),
 ('chinese|korean', 178657),
 ('arabic', 107944),
 ('greek', 94229),
 ('japanese', 88886),
 ('balkan', 84376),
 ('vietnamese', 80968),
 ('turkish', 42159),
 ('arabic|indian', 27677),
 ('arabic|indian|turkish', 21392),
 ('arabic|turkish', 14142)]